In [ ]:
!pip install dgl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 50.6 MB/s eta 0:00:00


### Dependencies

In [ ]:
import torch
import dgl
from collections import namedtuple
import dgl.function as fn
from copy import deepcopy as dc
import random
import time
from time import time
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
from dgl import DGLGraph

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


## Graph

In [ ]:
cost = torch.tensor([300, 300, 300, 300, 300])

features = torch.tensor([cost.cpu().detach().numpy()])
print(type(features), len(features), features)
print(type(features[0]), len(features[0]), features[0])

maxb = 1000
src_ids = torch.tensor([0, 1, 2, 3])
dst_ids = torch.tensor([1, 2, 3, 4])

g = dgl.graph((src_ids, dst_ids), num_nodes=5)
g = dgl.add_self_loop(g)
g.ndata['feat'] = torch.tensor([[300, 0],[300, 0],[300, 0],[300, 0],[300, 0]])

print(g.ndata)
print(g.nodes())
print(g.edges())
print(g.ndata['feat'].shape)


<class 'torch.Tensor'> 1 tensor([[300, 300, 300, 300, 300]])
<class 'torch.Tensor'> 5 tensor([300, 300, 300, 300, 300])
{'feat': tensor([[300,   0],
        [300,   0],
        [300,   0],
        [300,   0],
        [300,   0]])}
tensor([0, 1, 2, 3, 4])
(tensor([0, 1, 2, 3, 0, 1, 2, 3, 4]), tensor([1, 2, 3, 4, 0, 1, 2, 3, 4]))
torch.Size([5, 2])


<ipython-input-3-f6e5f576eaf1>:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  features = torch.tensor([cost.cpu().detach().numpy()])


## Model
https://discuss.dgl.ai/t/node-regression-in-heterograph/2492/6

In [ ]:

class RGCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes, seed):
        super(RGCN, self).__init__()
        self.seed = seed
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, h_feats)
        self.fc3 = nn.Linear(h_feats, num_classes)
        self.num_classes = num_classes

    def forward(self, g, in_feat):
        inp = len(in_feat) 
        print("0", inp)
        h = self.conv1(g, in_feat)
        print("1", h, h.shape)
        h = F.relu(h)
        print("2", h, h.shape)
        h = self.conv2(g, h)
        print("3", h, h.shape)
        h = self.fc3(F.relu(h))
        print("4", h, h.shape)
        T = h.mean(dim=0)
        T = T.mean(dim=0)
        return T.reshape(inp, self.num_classes)


In [ ]:
# model = RGCN(g.ndata['feat'].shape[1], 16, 5, 0)

st1 = torch.tensor([1, 0, 0, 0, 0])
st2 = torch.tensor([1, 1, 0, 0, 0])

f1 = g.ndata['feat']
f1[:, 1] = st1

f2 = g.ndata['feat']
f2[:, 1] = st2

# model = RGCN(f1.shape[1], 16, 5, 0)
# print(st1, f1, f1.shape)
# action = model(g, f1)
# print(action)

# # f = torch.tensor()


f2 = torch.tensor([[[300,   1],
        [300,   1],
        [300,   0],
        [300,   0],
        [300,   0]]])

f1 = np.array([[300,   1],
        [300,   1],
        [300,   0],
        [300,   0],
        [300,   0]])
f3 = torch.tensor([f1 ,f1])
print(f3, f3.shape)
# print(st2, f2, f2.shape)
model = RGCN(f3.shape[2], 16, 5, 0)
action = model(g, f3)
print(action)

tensor([[[300,   1],
         [300,   1],
         [300,   0],
         [300,   0],
         [300,   0]],

        [[300,   1],
         [300,   1],
         [300,   0],
         [300,   0],
         [300,   0]]]) torch.Size([2, 5, 2])
0 2


RuntimeError: ignored

## Agent

In [ ]:
import numpy as np
import random
from collections import namedtuple, deque

import torch
import torch.nn.functional as F
import torch.optim as optim

BUFFER_SIZE = int(1e5)  # replay buffer size
# BATCH_SIZE = 64         # minibatch size
BATCH_SIZE = 2         # minibatch size
GAMMA = 0.99            # discount factor
TAU = 1e-3              # for soft update of target parameters
LR = 5e-4               # learning rate 
# UPDATE_EVERY = 4        # how often to update the network
UPDATE_EVERY = 1       # how often to update the network

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Agent():
    """Interacts with and learns from the environment."""

    def __init__(self, state_size, action_size, seed):
        """Initialize an Agent object.
        
        Params
        ======
            state_size (int): dimension of each state
            action_size (int): dimension of each action
            seed (int): random seed
        """
        self.state_size = state_size
        self.action_size = action_size
        print(g, features)
        self.seed = random.seed(seed)

        # Q-Network
        # model = RGCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)
        self.qnetwork_local = RGCN(g.ndata['feat'].shape[1], 16, action_size, seed).to(device)
        self.qnetwork_target = RGCN(g.ndata['feat'].shape[1], 16, action_size, seed).to(device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=LR)

        # Replay memory
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, seed)
        # Initialize time step (for updating every UPDATE_EVERY steps)
        self.t_step = 0
    
    def step(self, state, action, reward, next_state, done):
        # Save experience in replay memory
        # print("Add to memory : ", state, action, next_state, reward)
        self.memory.add(state, action, reward, next_state, done)
        
        # Learn every UPDATE_EVERY time steps.
        self.t_step = (self.t_step + 1) % UPDATE_EVERY
        if self.t_step == 0:
            # If enough samples are available in memory, get random subset and learn
            if len(self.memory) > BATCH_SIZE:
                experiences = self.memory.sample()
                self.learn(experiences, GAMMA)

    def act(self, env, state, eps=0.):
        """Returns actions for given state as per current policy.
        
        Params
        ======
            state (array_like): current state
            eps (float): epsilon, for epsilon-greedy action selection
        """
        node_state = state[: -1 ]
        valid_action = []
        for i in range(len(node_state)):
            if(node_state[i] == 0 and env.cost[i] <= env.max_budget):
                valid_action.append(True)
            else:
                valid_action.append(False)

        valid_action_arg = []
        for i in range(len(valid_action)):
            if valid_action[i] == True:
                valid_action_arg.append(i)


        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        self.qnetwork_local.eval()
        with torch.no_grad():
            new_features = g.ndata['feat']
            # print(state, self.state_size,  state[:,: self.state_size])
            new_features[:, 1] = state[:,: self.state_size]
            action_values = self.qnetwork_local(g, new_features)
            # print("training local model", state, action_values)
        self.qnetwork_local.train()

        mn = -1e9
        for i in range(len(valid_action)):
            if(valid_action[i] == False):
                action_values[0][i] = mn

        # print("action_values", action_values)
        # print("valid_idx", valid_action_arg)
        
        # Epsilon-greedy action selection
        if random.random() > eps:
            # picks action with maximum value attached to it 
            choice = np.argmax(action_values.cpu().data.numpy())
            if choice == mn : 
                return -1

            # print("non greedy : ", choice)
            return choice
        else:
            # randomly pick any action from the values that are not already chosen 
            # return random.choice(np.arange(self.action_size))
            if len(valid_action_arg) == 0 :
                return -1
                
            choice = random.choice(valid_action_arg)
            # print("greedy : ", choice)
            return choice

    def learn(self, experiences, gamma):
        """Update value parameters using given batch of experience tuples.

        Params
        ======
            experiences (Tuple[torch.Tensor]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """
        states, actions, rewards, next_states, dones = experiences
        print("states", states)
        print("actions", actions)
        print("rewards", rewards)
        print("next_states", next_states)
        print("dones", dones)
              
        # Q_targets_next = self.qnetwork_target(g, features).detach().max(1)[0].unsqueeze(1)
        # # Compute Q targets for current states 
        # Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))

        Q_targets = []
        for i in range(len(next_states)) :
            ns = next_states[i]
            f = g.ndata['feat']
            print("0", f, ns, ns[: self.state_size])
            f[:, 1] = ns[: self.state_size]
            print("1", f)
            q_target_next = self.qnetwork_target(g, f).detach().max(1)[0].unsqueeze(1)
            print("2", q_target_next)
            q_target = rewards[i] + (gamma * q_target_next * (1 - dones[i]))
            Q_targets.append(q_target)

        # Q_expected = self.qnetwork_local(g, features).gather(1, actions)
        Q_expected = []
        for i in range(len(states)):
            s = states[i]
            f = g.ndata['feat']
            f[:, 1] = s[: self.state_size]
            action_values = self.qnetwork_local(g, f)
            print("3", action_values)
            choice = np.argmax(action_values.cpu().data.numpy())
            q_expected = action_values[:, choice]
            Q_expected.append(q_expected)


        Q_expected = np.array(Q_expected)
        Q_targets = np.array(Q_targets)

        Q_expected = torch.from_numpy(Q_expected)
        Q_targets = torch.from_numpy(Q_targets)
        
        # Compute loss
        loss = F.mse_loss(Q_expected, Q_targets)
        # Minimize the loss
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # ------------------- update target network ------------------- #
        self.soft_update(self.qnetwork_local, self.qnetwork_target, TAU)                     

    def soft_update(self, local_model, target_model, tau):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target

        Params
        ======
            local_model (PyTorch model): weights will be copied from
            target_model (PyTorch model): weights will be copied to
            tau (float): interpolation parameter 
        """
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)


class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.

        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
            seed (int): random seed
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
  
        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

## Environment

In [ ]:
import torch
import dgl
from collections import namedtuple
import dgl.function as fn
from copy import deepcopy as dc
import random
import time
from time import time
from torch.utils.data import DataLoader
import numpy as np
from collections import deque

class IM(object):
    def __init__(self, max_budget, p, num_nodes, cost):
        self.max_budget = max_budget
        self.BUDGET = max_budget
        assert(p <= 1 and p >= 0)
        self.p = p
        self.num_nodes = num_nodes
        self.cost = cost

    def compute_reward(self, state):
        # reward is the number of additional nodes influenced
        reward = 0
        # each node has one chance to influence each neighbour
        new_influenced = state.detach().cpu().numpy().ravel()
        tot_influenced = state.detach().cpu().numpy().ravel()
        while((new_influenced == 1).sum() >= 1):
            # next = torch.full(
            #     (self.num_nodes, 1),
            #     0, 
            #     dtype = torch.long
            #     )
            next = np.zeros(self.num_nodes)
            for e in range(self.g.number_of_edges()):
                # print(new_influenced[self.g.edges()[0][e]])
                if((new_influenced[self.g.edges()[0][e]] == 1) and 
                   not(tot_influenced[self.g.edges()[1][e]] == 1 or new_influenced[self.g.edges()[1][e]] == 1)):
                    r = random.random()
                    if(r < self.p):
                        # node influenced
                        next[self.g.edges()[1][e]] = 1
                        reward += 1
            # print(new_influenced)
            tot_influenced = tot_influenced + new_influenced
            new_influenced = next
        return reward
     
    def step(self, action):
        reward, sol, done = self._take_action(action)
        
        ob = self._build_ob()
        self.sol = sol
        info = {"sol": self.sol}

        # need to convert ob to ndarray from tensor
        ob = ob.detach().cpu().numpy().ravel()
        next_state = np.copy(ob)
        return next_state, reward, done, info
    
    def _take_action(self, action):
        r1, r2 = 0, 0
        num_iter = 100
        for i in range(num_iter):
            r1 += self.compute_reward(self.x[:-1])
        if(self.x[action] == 0 and self.cost[action] <= self.max_budget):
            self.x[action] = 1
            self.x[-1] -= self.cost[action]
            self.max_budget -= self.cost[action]
        # write code for else case 
        next_sol = 0
        for i in range(num_iter):
            r2 += self.compute_reward(self.x[:-1])
        done = self._check_done()
        return (r2 - r1)/num_iter, next_sol, done

    def _check_done(self): 
        inactive = (self.x[:-1] == 0).type(torch.float)
        # print(inactive)
        self.g.ndata['h'] = inactive
        not_selected = dgl.sum_nodes(self.g, 'h')
        self.g.ndata.pop('h')
        done = (not_selected == 0) or (self.max_budget <= 0)
        return done
                
    def _build_ob(self):
        ob_x = self.x
        # ob = torch.cat([ob_x], dim = 2)
        # return ob
        return ob_x
    
    # using num_samples = 1 as of now 
    def register(self, g, num_samples = 1):
        self.g = g
        self.g.set_n_initializer(dgl.init.zero_initializer)
        t = torch.full((self.num_nodes, 1), 0, dtype=torch.float16)
        # torch.full(
            #     (self.num_nodes, 1),
            #     0, 
            #     dtype = torch.long
            #     )
        self.x = torch.cat((t, torch.tensor([[self.max_budget]])), 0)
        ob = self._build_ob()
        return ob


    def reset(self):
        state = np.zeros(self.num_nodes + 1)
        state[-1] = self.BUDGET

        t = torch.full((self.num_nodes, 1), 0, dtype=torch.float16)
        self.max_budget = self.BUDGET
        self.x = torch.cat((t, torch.tensor([[self.max_budget]])), 0)

        return np.array(state)

## Main 

In [ ]:
env = IM(maxb, 0.6, 5, cost)
ob = env.register(g)

print(g.nodes())
print(g.edges())
print(ob)

tensor([0, 1, 2, 3, 4])
(tensor([0, 1, 2, 3, 0, 1, 2, 3, 4]), tensor([1, 2, 3, 4, 0, 1, 2, 3, 4]))
tensor([[   0.],
        [   0.],
        [   0.],
        [   0.],
        [   0.],
        [1000.]], dtype=torch.float16)


In [ ]:
# agent = Agent(state_size=env.num_nodes + 1, action_size=env.num_nodes, seed=0)
agent = Agent(state_size=env.num_nodes , action_size=env.num_nodes, seed=0)

passed_state  = [1, 0, 0, 1, 0, 1000]
print(passed_state)

node_state = passed_state[: -1 ]
print(node_state, node_state == 1)

valid_idx = []
for i in range(len(node_state)):
    if(node_state[i] == 0):
        valid_idx.append(i)

print(valid_idx)


print(random.choice(np.array(valid_idx)))


### dqn

In [ ]:
def dqn(n_episodes=2000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start                    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        state = env.reset()
        score = 0
        for t in range(max_t):
            action = agent.act(env, state, eps)
            if action == -1:
                break 
            next_state, reward, done, _ = env.step(action)
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break 

        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 10 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=200.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
            break
    return scores

scores = dqn(2)


In [ ]:
scores = dqn(100)


In [ ]:
# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()